# Test runtime
### Test with IK checkers

**grasp sample 10**  
solve-mean: 1175.2 ms  
solve-median: 740.8 ms  
success rate: 98.0 %  

**grasp sample 20**  
solve-mean: 1190.6 ms  
solve-median: 778.4 ms  
success rate: 96.5 %  

**grasp sample 30**  
solve-mean: 1323.0 ms  
solve-median: 837.4 ms  
success rate: 99.0 %  

### Test without IK checkers

**grasp sample 10**  
solve-mean: 1998.9 ms  
solve-median: 1241.1 ms  
success rate: 88.0 %  

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt
from pkg.controller.combined_robot import *

# params

In [2]:
ROBOT_TYPE = RobotType.indy7

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    
else:
    raise(NotImplementedError("Case not defined for {}".format(ROBOT_TYPE)))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [3]:
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [4]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.72,1.52,0.01), (0.25,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.1,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(np.pi/2,0,np.pi/6), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

box2 = gscene.create_safe(GEOTYPE.BOX, "box2", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(np.pi,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

box1.permute_axis_match_z_link()
box2.permute_axis_match_z_link()

##### create PlanningScene

In [5]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [6]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [7]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=None)

In [8]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.SPHERE, TOOL_NAME, link_name=TOOL_LINK, 
                                dims=(0.01,0.01,0.01), center=TOOL_XYZ, rpy=(-np.pi/2,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=False, fixed=True, collision=False)

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
obs_obj = pscene.create_subject(oname="box2", gname="box2", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box2, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box2, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box2, [0,0,-0.026], [0,0,0])})

## Pipeline

In [12]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [13]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [14]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

   Use a production WSGI server instead.
('floor', 'floor')
 * Debug mode: off


## import planners

In [15]:

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)

checkers_all = [rcheck, gcheck]

# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]

# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

## Test pybullet

In [16]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))

In [17]:
from convert_pscene import *
from plan_rnb import *

#### load pybullet, convert scene

In [18]:
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)
robot_body, body_names, movable_bodies = pscene_to_pybullet(
    pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
print('Objects:', body_names)
saver = WorldSaver()

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
('Objects:', {1L: 'goal', 2L: 'floor', 3L: 'wall', 4L: 'box1', 5L: 'box2'})


### Test with IK checkers

In [19]:
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)

In [20]:
log_wFeas = []
for _ in range(20):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=checkers_all,
                                          tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF, max_time=10)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_wFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p0), ('AtPose', 4L, p0), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p0, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p1), ('AtPose', 5L, p1), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p1, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q0), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-traj-pose]
Setting negate=True for stream [test-cfree-pose-pose]
Set

q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.0146626077339285, -0.3015863406665864, -2.598228648935811, -1.6563728233785395, -0.13398847441062087, -1.0103834163845504)
go on
q_grasp: (0.6594326623980555, -0.46004421391909794, -2.4547415278079057, -2.537043367930925, -0.06834403758773078, -0.47309424972067304)
iter=inf, outs=1) inverse-kinematics:(4, p0, g3)->[(q7, c2)]
iter=inf, outs=1) test-cfree-traj-pose:(c2, 4, p0)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c2, 5, p1)->[()]
q_approach: (-0.8350851291792754, -0.9139609046051812, -1.100456626985209, 0.3160821114352755, -1.6579873206729985, 0.5096033623563021)
obstacles: [4L, 1, 2, 3]
q_approach: (-1.369763254697037, -1.9914345611932698, 1.189293277779622, -3.036456143787027, 2.951642340761185, -3.0108962298950397)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: Non

q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.3251844213683215, 1.0867668750298245, 0.8991881949651105, 0.13541471520713513, 1.6746496566693307, 3.385816478542372)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.3337651187596193, -1.0785631651962557, -0.6570479632706138, 0.36477215911066657, -1.8004692244045974, 0.7607902528039359)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
iter=inf, outs=0) inverse-kinematics:(4, p9, g1)->[]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.2981196214035875, -0.988533483149529, -0.827048036698054, 0.3119317441617173, -1.6498776277868863, 0.7578670314638591)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.7889833981025713, -1.836822522

q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.7614842515458613, 2.0028552741830032, -1.3123608164844287, 0.08139760412939544, 2.9868920785380806, -3.041215143477496)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (2.3206835114485576, 0.847846373407811, 1.2014000154319397, -2.856587412336344, -1.5540944631075828, 0.5101272182097125)
obstacles: [4L, 1, 2, 3]
q_approach: (1.7591353412930741, 0.8467431469957676, 1.3638720813621692, 0.010773010874975208, 1.4671236403858496, -3.12485269406721)
obstacles: [4L, 1, 2, 3]
iter=inf, outs=0) inverse-kinematics:(4, p3, g4)->[]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (2.1535300809278084, 0.6247422243599988, 1.4960022616368962, -2.5162794591480657, -1.050478959536551, 1.3540098395685485)
obstacles: [4L, 1, 2, 3]
q_approach: (-0.9748212510384878, -0.6217026525342778, -1.5010899134259421, 0.6276736753633373, -1.04

iter=inf, outs=1) plan-free-motion:(q0, q10)->[(c11)]
iter=inf, outs=0) plan-free-motion:(q9, q9)->[]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-0.4432339393754742, -2.090385168959842, 1.6069572022706766, -0.572524384447804, -2.594326122400628, -2.038330420141183)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-0.4421779995495588, -2.0928555929646206, 1.6092615090141709, -0.5722513859239126, -2.5943135429906796, -2.036893958379041)
obstacles: [4L, 1, 2, 3]
q_approach: (-0.4368264038625856, -0.4962323422771997, -2.147403096032832, -0.540960796080987, -0.5783996092621977, -1.0625393655909765)
go on
q_grasp: (-0.5194274906568978, -0.5768373628200364, -2.242117403372354, -0.7646951745578817, -0.4187522929269078, -0.8949387914607331)
obstacles: [4L, 1, 2, 3]
q_approach: (-0.6091109651409661, -2.136724514964831, 2.3070283775347975, 1.148198426940281, 2.829713399504632, -0.5787512554849596)
obstacles: [4L,

...

q_approach: None
obstacles: [4L, 1, 2, 3]
iter=inf, outs=0) inverse-kinematics:(4, p87, g46)->[]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.2106535161149423, -2.035447007517007, 2.0147266477000536, 2.5261789647627113, -3.019948709347696, -3.223673388717445)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.2053982951431415, -2.0380323984008077, 2.016243665003946, 2.505944588840194, -3.0204235434431905, 3.0443710582361088)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.8524482786056105, 0.5681071495612369, 1.77466473709149, 0.07673802478779194, 0.925223839156987, -2.7361542478893837)
go on
q_grasp: (1.8603630302024698, 0.6414033774412423, 1.8289155688604872, 0.0861932225967816, 0.7981099201186175, -2.7422399987771597)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.090
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.198
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g17), inverse-kinematics:(4, p88, #g17)->(#q69, #t4814), test-cfree-traj-pose:(#t4814, 4, p88)->(), sample-pose:(4, 1)->(#p27), inverse-kinematics:(4, #p27, #g17)->(#q70, #t4815), test-cfree-traj-pose:(#t4815, 5, p89)->(), test-cfree-traj-pose:(#t4814, 5, p89)->(), test-cfree-pose-pose:(4, #p27, 5, p89)->(), test-cfree-approach-pose:(4, p88, #g17, 5, p89)->(), test-cfree-approach-pose:(4, #p27, #g17, 5, p89)->(), plan-free-motion:(#q70, q129)->(#t4856), plan-free-motion:(q129, #q69)->(#t4858), plan-holding-motion:(#q69, #q70, 4, #g17)->(#t4857)]
Action plan (5, 0.000): [move_free(q129, #q69, #t4858), pick(4, p88, #g17, #q69, #t4814), move_holding(#q69, #q70, 4, #g17, #t4857), place(4, #p27, #g17, #q70, #t4815), move_free(#q70, q129, #t4856)]
iter=0, outs=1) sample-grasp:(4)->[(g49)]
q_approach: None
obstac

Wrote: statistics/py2/kuka-tamp.pkl
         1674258 function calls (1607183 primitive calls) in 0.932 seconds

   Ordered by: internal time
   List reduced from 1507 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      468    0.156    0.000    0.156    0.000 /home/rnb/Projects/pddlstream/pddlstream/algorithms/../../FastDownward/builds/release64/bin/translate/pddl/pddl_types.py:66(get_atom)
        3    0.121    0.040    0.122    0.041 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
   107748    0.066    0.000    0.083    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.053    0.026    0.208    0.104 /usr/lib/python2.7/pickle.py:851(load)
  53952/1    0.039    0.000    0.094    0.094 /usr/lib/python2.7/pickle.py:269(save)
   215772    0.038    0.000    0.038    0.000 {method 'read' of 'file' objects}
        2    0.036    0.018    0.036    0.018 {posix.read}
   107716    0.031    0.000    0.043    

Attempt: 1 | Results: 48 | Depth: 1 | Success: False | Time: 0.141
Attempt: 2 | Results: 210 | Depth: 1 | Success: False | Time: 0.347
Attempt: 3 | Results: 338 | Depth: 0 | Success: True | Time: 0.690
Stream plan (25, 22, 0.001): [sample-grasp:(4)->(g52), sample-pose:(4, 2)->(#p30), inverse-kinematics:(4, #p30, g52)->(#q78, #t5010), test-cfree-traj-pose:(#t5010, 5, p95)->(), sample-grasp:(4)->(g53), inverse-kinematics:(4, p94, g53)->(#q76, #t5008), test-cfree-traj-pose:(#t5008, 5, p95)->(), test-cfree-traj-pose:(#t5008, 4, p94)->(), sample-pose:(4, 1)->(p97), inverse-kinematics:(4, p97, g52)->(#q77, #t5009), test-cfree-traj-pose:(#t5009, 5, p95)->(), inverse-kinematics:(4, #p30, g53)->(#q75, #t5007), test-cfree-traj-pose:(#t5007, 5, p95)->(), test-cfree-traj-pose:(#t5010, 4, #p30)->(), test-cfree-pose-pose:(4, p97, 5, p95)->(), test-cfree-pose-pose:(4, #p30, 5, p95)->(), test-cfree-approach-pose:(4, p97, g52, 5, p95)->(), test-cfree-approach-pose:(4, p94, g53, 5, p95)->(), test-cfree-

Attempt: 1 | Results: 414 | Depth: 0 | Success: True | Time: 0.468
Stream plan (25, 17, 0.001): [sample-grasp:(4)->(g53), inverse-kinematics:(4, p94, g53)->(q138, c131), test-cfree-traj-pose:(c131, 5, p95)->(), sample-grasp:(4)->(g52), sample-pose:(4, 1)->(p96), inverse-kinematics:(4, p96, g52)->(q136, c129), test-cfree-traj-pose:(c129, 5, p95)->(), test-cfree-traj-pose:(c131, 4, p94)->(), sample-pose:(4, 3)->(#p31), inverse-kinematics:(4, #p31, g53)->(#q79, #t5175), test-cfree-traj-pose:(#t5175, 5, p95)->(), inverse-kinematics:(4, #p31, g52)->(#q80, #t5176), test-cfree-traj-pose:(#t5176, 5, p95)->(), test-cfree-traj-pose:(#t5176, 4, #p31)->(), test-cfree-pose-pose:(4, p96, 5, p95)->(), test-cfree-pose-pose:(4, #p31, 5, p95)->(), test-cfree-approach-pose:(4, p94, g53, 5, p95)->(), test-cfree-approach-pose:(4, p96, g52, 5, p95)->(), test-cfree-approach-pose:(4, #p31, g53, 5, p95)->(), test-cfree-approach-pose:(4, #p31, g52, 5, p95)->(), plan-free-motion:(q136, q135)->(#t5615), plan-free

In [21]:
print(gtimer)

time_array = np.array(gtimer.timelist_dict["solve"])
success_array = np.array(log_wFeas)<1
print("solve-mean: {} ms".format(np.round(np.mean(time_array[np.where(success_array)[0]]), 1)))
print("solve-median: {} ms".format(np.round(np.median(time_array[np.where(success_array)[0]]), 1)))
print("success rate: {} %".format(np.mean(success_array)*100))

solve: 	36830.2 ms/20 = 1841.5 ms (703.19/5481.78)
sample_grasps_4: 	78.7 ms/20 = 3.9 ms (2.878/9.41)
get_stable_4_1: 	40.1 ms/45 = 0.9 ms (0.499/2.171)
ik_fn: 	13152.8 ms/226 = 58.2 ms (0.695/270.329)
check_feas: 	748.9 ms/226 = 3.3 ms (0.685/8.933)
ReachChecker: 	202.1 ms/226 = 0.9 ms (0.233/2.528)
GraspChecker: 	388.8 ms/212 = 1.8 ms (0.533/4.853)
get_stable_4_2: 	14.4 ms/16 = 0.9 ms (0.489/1.335)
get_stable_4_3: 	0.0 ms/5 = 0.0 ms (0.003/0.006)
free_motion_gen: 	2187.3 ms/45 = 48.6 ms (4.008/86.75)
holding_motion_gen: 	1052.8 ms/23 = 45.8 ms (27.388/78.251)

solve-mean: 1841.5 ms
solve-median: 1100.7 ms
success rate: 100.0 %


### Test without IK checkers

In [22]:
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)

In [23]:
log_woFeas = []
for _ in range(20):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=[],
                                          tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF, max_time=10)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_woFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q87), ('AtConf', q87), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p91), ('AtPose', 4L, p91), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p91, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p92), ('AtPose', 5L, p92), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p92, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q87), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-t

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.082
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.187
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g21), sample-pose:(4, 1)->(#p23), inverse-kinematics:(4, #p23, #g21)->(#q48, #t1843), test-cfree-traj-pose:(#t1843, 5, p95)->(), inverse-kinematics:(4, p94, #g21)->(#q47, #t1842), test-cfree-traj-pose:(#t1842, 5, p95)->(), test-cfree-traj-pose:(#t1842, 4, p94)->(), test-cfree-pose-pose:(4, #p23, 5, p95)->(), test-cfree-approach-pose:(4, p94, #g21, 5, p95)->(), test-cfree-approach-pose:(4, #p23, #g21, 5, p95)->(), plan-free-motion:(#q48, q91)->(#t1884), plan-free-motion:(q91, #q47)->(#t1885), plan-holding-motion:(#q47, #q48, 4, #g21)->(#t1886)]
Action plan (5, 0.000): [move_free(q91, #q47, #t1885), pick(4, p94, #g21, #q47, #t1842), move_holding(#q47, #q48, 4, #g21, #t1886), place(4, #p23, #g21, #q48, #t1843), move_free(#q48, q91, #t1884)]
iter=0, outs=1) sample-grasp:(4)->[(g57)]
iter=0, outs=1) sample-pose

q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-2.71389979194755, -0.3047511583700157, -2.7157285208676014, -2.4840238282195815, -0.7004655246149153, 2.867543012046386, 1.7546519414316757)
go on
q_grasp: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
iter=inf, outs=0) inverse-kinematics:(4, p98, g60)->[]
iter=3, outs=1) sample-grasp:(4)->[(g61)]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-0.17874639573274542, 1.1374769517538854, 1.2038521895773393, -1.8781811832346804, -0.8239896425998763, 1.7440369187911269, 1.6726923383992254)
go on
q_grasp: (-0.17824263677035704, 1.1769515945486235, 1.1425073567821773, -1.9404756615773726, -0.8847809822390776, 1.8295088103449486, 1.7021559584471642)
obstacles: [4L, 1, 2, 3]
q_approach: (1.2344512348608219, 0.5848336302627009, -0.40251068241000093, -1.9249355941897155, 0.39381119134

iter=inf, outs=1) plan-free-motion:(q103, q104)->[(c122)]
iter=inf, outs=1) plan-free-motion:(q105, q103)->[(c123)]
iter=inf, outs=1) plan-holding-motion:(q104, q105, 4, g63)->[(c124)]
Summary: {complexity: 2, cost: 0.000, evaluations: 36, iterations: 3, length: 2, run_time: 0.508, sample_time: 0.253, search_time: 0.255, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 6926 | p_success: 0.963 | overhead: 0.001
External: sample-grasp | n: 8985 | p_success: 0.994 | overhead: 0.002
External: inverse-kinematics | n: 30210 | p_success: 0.217 | overhead: 0.048
External: plan-free-motion | n: 5910 | p_success: 0.891 | overhead: 0.126
External: plan-holding-motion | n: 2268 | p_success: 0.991 | overhead: 0.059
External: test-cfree-pose-pose | n: 3174 | p_success: 1.000 | overhead: 0.001
External: test-cfree-approach-pose | n: 6246 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 11584 | p_success: 0.937 | ov

Wrote: statistics/py2/kuka-tamp.pkl
         1590542 function calls (1526913 primitive calls) in 0.740 seconds

   Ordered by: internal time
   List reduced from 1475 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.111    0.037    0.112    0.037 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
    99912    0.058    0.000    0.074    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.049    0.024    0.188    0.094 /usr/lib/python2.7/pickle.py:851(load)
  50035/1    0.041    0.000    0.102    0.102 /usr/lib/python2.7/pickle.py:269(save)
   200100    0.034    0.000    0.034    0.000 {method 'read' of 'file' objects}
        2    0.033    0.016    0.033    0.016 {posix.read}
    99880    0.029    0.000    0.040    0.000 /usr/lib/python2.7/pickle.py:1184(load_append)
    49970    0.016    0.000    0.027    0.000 /usr/lib/python2.7/pickle.py:443(save_int)
   100068    0.016    0.000    0.016  

...

q_grasp: (-2.0538350424986493, -0.5830429550562455, 2.2277479746688034, -2.5583152115947767, 0.42733517934027093, 2.687002489690382, -0.5467903774847263)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.6139270739749965, -0.9596559501224461, 1.849961342962375, -2.497406409176309, 0.94285588875871, 2.291241618599881, -0.8979198406251866)
go on
q_grasp: (-1.7316038753641942, -0.9913853735542699, 2.003505928590355, -2.487059082349431, 1.0553331320964408, 2.3883094249439907, -0.9631244165346581)
obstacles: [4L, 1, 2, 3]
q_approach: (-1.5811502429518018, -1.0722248708016484, 1.821429735426037, -2.484833037732631, 1.0409482813521285, 2.2028276744654884, -0.9566268740934416)
go on
q_grasp: (-1.6781610929188406, -1.102265577488691, 1.9621046565388636, -2.475277284580131, 1.1541904740049205, 2.2911270991882415, -1.0157750072290825)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.3686334060017367, 0.9660167651607444, 1.38373227

Attempt: 1 | Results: 358 | Depth: 1 | Success: False | Time: 0.514
Attempt: 2 | Results: 715 | Depth: 1 | Success: False | Time: 1.039
Attempt: 3 | Results: 1023 | Depth: 0 | Success: True | Time: 1.877
Stream plan (37, 24, 0.001): [sample-grasp:(4)->(g100), sample-pose:(4, 2)->(p182), inverse-kinematics:(4, p182, g100)->(q287, c229), test-cfree-traj-pose:(c229, 5, p174)->(), sample-pose:(4, 1)->(p175), inverse-kinematics:(4, p175, g100)->(q275, c227), test-cfree-traj-pose:(c227, 5, p174)->(), test-cfree-traj-pose:(c229, 4, p182)->(), sample-grasp:(4)->(g102), inverse-kinematics:(4, p173, g102)->(q286, c228), test-cfree-traj-pose:(c228, 4, p173)->(), test-cfree-traj-pose:(c228, 5, p174)->(), sample-grasp:(4)->(g101), inverse-kinematics:(4, p182, g101)->(#q121, #t6117), test-cfree-traj-pose:(#t6117, 5, p174)->(), sample-pose:(4, 2)->(p184), inverse-kinematics:(4, p184, g101)->(#q119, #t6115), test-cfree-traj-pose:(#t6115, 4, p184)->(), inverse-kinematics:(4, p184, g102)->(#q120, #t6116

iter=inf, outs=1) test-cfree-traj-pose:(c232, 4, p183)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c233, 4, p185)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c237, 4, p182)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c238, 4, p185)->[()]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.7670099777313784, -0.964890947754169, -2.0569891635825335, -2.2526552304380227, -0.5814631717087596, 2.620453213748669, 2.6748264316524852)
go on
q_grasp: (1.899787074434508, -1.017240874906355, -2.169921029466434, -2.2171850636826775, -0.6293376261911169, 2.7141250411311564, 2.712016555151987)
iter=inf, outs=1) inverse-kinematics:(4, p184, g103)->[(q317, c241)]
iter=inf, outs=1) test-cfree-traj-pose:(c241, 5, p174)->[()]
q_approach: (-1.464307500064385, -0.7558832918518746, 2.5278778

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.092
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.200
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g39), inverse-kinematics:(4, p187, #g39)->(#q124, #t6812), test-cfree-traj-pose:(#t6812, 4, p187)->(), test-cfree-traj-pose:(#t6812, 5, p188)->(), sample-pose:(4, 1)->(#p55), inverse-kinematics:(4, #p55, #g39)->(#q125, #t6813), test-cfree-traj-pose:(#t6813, 5, p188)->(), test-cfree-pose-pose:(4, #p55, 5, p188)->(), test-cfree-approach-pose:(4, p187, #g39, 5, p188)->(), test-cfree-approach-pose:(4, #p55, #g39, 5, p188)->(), plan-free-motion:(#q125, q323)->(#t6855), plan-free-motion:(q323, #q124)->(#t6856), plan-holding-motion:(#q124, #q125, 4, #g39)->(#t6854)]
Action plan (5, 0.000): [move_free(q323, #q124, #t6856), pick(4, p187, #g39, #q124, #t6812), move_holding(#q124, #q125, 4, #g39, #t6854), place(4, #p55, #g39, #q125, #t6813), move_free(#q125, q323, #t6855)]
iter=0, outs=1) sample-grasp:(4)->[(g105)]
q

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.085
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.223
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g40), inverse-kinematics:(4, p190, #g40)->(#q126, #t6871), test-cfree-traj-pose:(#t6871, 5, p191)->(), test-cfree-traj-pose:(#t6871, 4, p190)->(), sample-pose:(4, 1)->(#p56), inverse-kinematics:(4, #p56, #g40)->(#q127, #t6872), test-cfree-traj-pose:(#t6872, 5, p191)->(), test-cfree-pose-pose:(4, #p56, 5, p191)->(), test-cfree-approach-pose:(4, p190, #g40, 5, p191)->(), test-cfree-approach-pose:(4, #p56, #g40, 5, p191)->(), plan-free-motion:(q328, #q126)->(#t6913), plan-free-motion:(#q127, q328)->(#t6915), plan-holding-motion:(#q126, #q127, 4, #g40)->(#t6914)]
Action plan (5, 0.000): [move_free(q328, #q126, #t6913), pick(4, p190, #g40, #q126, #t6871), move_holding(#q126, #q127, 4, #g40, #t6914), place(4, #p56, #g40, #q127, #t6872), move_free(#q127, q328, #t6915)]
iter=0, outs=1) sample-grasp:(4)->[(g107)]
q

In [22]:
print(gtimer)

time_array = np.array(gtimer.timelist_dict["solve"])
success_array = np.array(log_woFeas)<1
print("solve-mean: {} ms".format(np.round(np.mean(time_array[np.where(success_array)[0]]), 1)))
print("solve-median: {} ms".format(np.round(np.median(time_array[np.where(success_array)[0]]), 1)))
print("success rate: {} %".format(np.mean(success_array)*100))

solve: 	114004.4 ms/20 = 5700.2 ms (944.096/10438.012)
sample_grasps_4: 	87.0 ms/20 = 4.3 ms (2.755/12.515)
get_stable_4_1: 	426.4 ms/441 = 1.0 ms (0.395/2.828)
ik_fn: 	64947.5 ms/4492 = 14.5 ms (1.353/231.827)
check_feas: 	15598.1 ms/4492 = 3.5 ms (1.345/10.35)
ReachChecker: 	4004.7 ms/4492 = 0.9 ms (0.35/3.058)
GraspChecker: 	8115.1 ms/4492 = 1.8 ms (0.59/6.77)



NameError: name 'log_woFeas' is not defined

## play plan

In [23]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.001)